In [19]:
import torch
import torch.nn as nn
import numpy as np
from sklearn import datasets
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import math

# Prepare Data

In [20]:
bc=datasets.load_breast_cancer()
X,y=bc.data,bc.target

In [21]:
X.shape,y.shape

((569, 30), (569,))

In [22]:
n_samples,n_features=X.shape

X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=1234)

In [23]:
X_train.shape,y_train.shape

((455, 30), (455,))

In [24]:
X_test.shape,y_test.shape

((114, 30), (114,))

In [25]:
# Scaling
sc=StandardScaler()
X_train=sc.fit_transform(X_train)
X_test=sc.transform(X_test)

X_train=torch.from_numpy(X_train.astype(np.float32))
X_test=torch.from_numpy(X_test.astype(np.float32))
y_train=torch.from_numpy(y_train.astype(np.float32))
y_test=torch.from_numpy(y_test.astype(np.float32))


In [26]:
X_train.shape,y_train.shape

(torch.Size([455, 30]), torch.Size([455]))

In [27]:
X_test.shape,y_test.shape

(torch.Size([114, 30]), torch.Size([114]))

In [28]:
y_train=y_train.view(y_train.shape[0],1)
y_test=y_test.view(y_test.shape[0],1)

y_train.shape,y_test.shape

(torch.Size([455, 1]), torch.Size([114, 1]))

In [29]:
X_train.shape,X_test.shape

(torch.Size([455, 30]), torch.Size([114, 30]))

# Build Model

In [30]:
class LogisticRegression(nn.Module):
    def __init__(self,n_input_features):
        super(LogisticRegression,self).__init__()
        self.linear=nn.Linear(n_input_features,1)
        
    def forward(self,x):
        y_predicted=torch.sigmoid(self.linear(x))
        return y_predicted
    

In [33]:
model=LogisticRegression(n_features)
learning_rate=0.01

loss_fn=nn.BCELoss()
optimizer=torch.optim.SGD(model.parameters(),lr=learning_rate)

#Training loop
num_epochs=100

for epoch in range(num_epochs):
    #forward pass and loss
    y_pred=model(X_train)
    loss_val=loss_fn(y_pred,y_train)
    
    #backward pass
    loss_val.backward()
    
    #updates weights and bias
    optimizer.step()
    
    #zero gradients
    optimizer.zero_grad()
    
    if (epoch+1) % 10 == 0 :
        print(f"epoch {epoch+1} , loss = {loss_val.item():.4f}")
    

with torch.no_grad():
    y_test_pred=model(X_test)
    y_pred_classes=y_test_pred.round()
    print(y_test_pred.shape)
    accuracy=y_pred_classes.eq(y_test).sum() / float(y_test_pred.shape[0])
    print(f"Accuracy : {accuracy:.4f}")



    

epoch 10 , loss = 0.6191
epoch 20 , loss = 0.5051
epoch 30 , loss = 0.4345
epoch 40 , loss = 0.3862
epoch 50 , loss = 0.3508
epoch 60 , loss = 0.3235
epoch 70 , loss = 0.3018
epoch 80 , loss = 0.2839
epoch 90 , loss = 0.2688
epoch 100 , loss = 0.2559
torch.Size([114, 1])
Accuracy : 0.8860


In [35]:
y_pred[:10]

tensor([[0.6217],
        [0.7733],
        [0.6633],
        [0.3284],
        [0.8787],
        [0.1650],
        [0.9280],
        [0.8432],
        [0.8916],
        [0.8290]], grad_fn=<SliceBackward0>)

In [36]:
y_pred_classes[:10]

tensor([[1.],
        [1.],
        [0.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [0.],
        [0.]])